In [51]:
import os
import pandas as pd
from tqdm import tqdm
import emoji, re, time

import boto3
import pyarrow.parquet as pymysql
from dotenv import load_dotenv
from io import BytesIO
import io
from PIL import Image
import gc

from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel

In [2]:
import torch
print(torch.__version__)           
print(torch.version.cuda)          
print(torch.cuda.is_available())   
print(torch.cuda.get_device_name(0)) 

2.6.0+cu124
12.4
True
NVIDIA H100 80GB HBM3


In [3]:
load_dotenv('/home/flexmatch_ftp/ftp/multimodal_categorizer/config/.env')

aws_access_key = os.getenv("aws_accessKey")
aws_secret_key = os.getenv("aws_secretKey")
region_name='ap-northeast-2'

# S3 클라이언트 생성
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

In [18]:
img_path_df = pd.read_parquet("/home/flexmatch_ftp/ftp/multimodal_categorizer/all_user_image_data.parquet")
img_path = img_path_df['s3_path'].to_list()

In [10]:
# bucket_name = 'flexmatch-data'
# sample = img_path[0]
# sample_img = load_image_from_s3(bucket_name, sample)
# sample_img.show()

In [ ]:
# 이미지만 로딩 

bucket_name = 'flexmatch-data'

def load_image_from_s3(bucket, key):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    img = Image.open(io.BytesIO(obj['Body'].read()))
    return img.convert("RGB")

images = []
for key in tqdm(img_path[:10000]):  # 예시로 100장만
    try:
        img = load_image_from_s3(bucket_name, key)
        images.append(img)
    except Exception as e:
        print(f"Error loading {key}: {e}")

100%|██████████| 10000/10000 [41:10<00:00,  4.05it/s] 


In [ ]:
def combine_texts_for_clip(df):
    combined_texts = [
        f"{acnt_sub_nm}. {intro}. {txt}"
        for acnt_sub_nm, intro, txt in zip(
            df["acnt_sub_nm"],
            df["intro_txt"],
            df["media_text"]
        )
    ]
    return combined_texts

combined_texts = combine_texts_for_clip(img_path_df)
combined_texts

In [ ]:
# # 이미지만 처리한다고 했을 때

# model_name = "openai/clip-vit-base-patch32"
# # 이미지 resize/normalized 및 text token화 및 tensor 변환 -> CLIP이 이해할 수 있는 형태로 변환
# processor = CLIPProcessor.from_pretrained(model_name)

# # CLIP 원본 모델 -> 이미지 및 텍스트 임베딩 변환하는 용도
# model = CLIPModel.from_pretrained(model_name)

# inputs = processor(images=images, return_tensors="pt", padding=True, use_fast=True)
# with torch.no_grad():
#     image_features = model.get_image_features(**inputs)

# class CLIPMultimodalClassifier(nn.Module):
#     def __init__(self, model_name="openai/clip-vit-base-patch32", num_classes=5):
#         super().__init__()
#         self.clip = CLIPModel.from_pretrained(model_name)
#         self.fc = nn.Linear(self.clip.visual_projection.out_features * 2, num_classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, image, text):
#         outputs = self.clip.get_image_features(image), self.clip.get_text_features(text)
#         image_feat = outputs[0] / outputs[0].norm(p=2, dim=-1, keepdim=True)
#         text_feat = outputs[1] / outputs[1].norm(p=2, dim=-1, keepdim=True)
#         combined = torch.cat([image_feat, text_feat], dim=1)
#         logits = self.fc(combined)
#         return self.softmax(logits)

# # 예시 입력
# from transformers import CLIPProcessor
# from PIL import Image

# model = CLIPMultimodalClassifier(num_classes=3)
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# image = Image.open("example.jpg")
# text = ["싱그러운 봄 패션 리뷰"]
# inputs = processor(text=text, images=image, return_tensors="pt", padding=True)

# # forward
# pred = model(inputs["pixel_values"], inputs["input_ids"])
# print(pred)


embedding

In [ ]:
BUCKET_NAME = "flexmatch-data"
MODEL_NAME = "openai/clip-vit-base-patch32"
BATCH_SIZE = 32
NUM_WORKERS = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class S3StreamingDataset(Dataset):
    def __init__(self, df, bucket):
        self.df = df.reset_index(drop=True)
        self.bucket = bucket

    def __len__(self):
        return len(self.df)

    ## getitem 마다 s3.get_object() 호출
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        key = row["s3_path"]

        try:
            # 매번 S3에서 직접 읽기
            obj = s3_client.get_object(Bucket=self.bucket, Key=key)
            img = Image.open(BytesIO(obj["Body"].read())).convert("RGB")
        except Exception as e:
            print(f"[WARN] S3 read failed for {key}: {e}")
            img = Image.new("RGB", (224,224), (0,0,0))

        # 텍스트 필드 합치기
        text_parts = []
        for col in ["acnt_sub_nm", "intro_txt", "media_text"]:
            if col in row and pd.notna(row[col]):
                text_parts.append(str(row[col]).strip())
        combined_text = ". ".join(text_parts) if text_parts else ""

        return {"image": img, "text": combined_text, "s3_key": key}

## 이미지+텍스트를 묶어 배치로 전달 -> 이후에 processor에 넣을 수 있음
def collate_fn(batch):
    images = [item["image"] for item in batch]
    texts = [item["text"] for item in batch]
    keys = [item["s3_key"] for item in batch]
    return {"images": images, "texts": texts, "keys": keys}

## clip 모델/프로세서 로드
processor = CLIPProcessor.from_pretrained(MODEL_NAME)
model = CLIPModel.from_pretrained(MODEL_NAME).to(DEVICE)

def extract_clip_embeddings(df):
    dataset = S3StreamingDataset(df, BUCKET_NAME)
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        collate_fn=collate_fn,
    )

    all_image_embeds, all_text_embeds, all_keys = [], [], []

    for batch in tqdm(loader, desc="Extracting CLIP embeddings"):
        try:
            inputs = processor(
                text=batch["texts"],
                images=batch["images"],
                return_tensors="pt",
                padding=True,
                truncation=True,
            ).to(DEVICE)

            with torch.no_grad():
                outputs = model(**inputs)
                image_embeds = outputs.image_embeds
                text_embeds = outputs.text_embeds

            all_image_embeds.append(image_embeds.cpu())
            all_text_embeds.append(text_embeds.cpu())
            all_keys.extend(batch["keys"])

            # 메모리 해제
            del inputs, outputs, image_embeds, text_embeds
            gc.collect()
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"[ERROR] Batch failed: {e}")
            continue

    image_tensor = torch.cat(all_image_embeds, dim=0)
    text_tensor = torch.cat(all_text_embeds, dim=0)

    return image_tensor, text_tensor, all_keys

### ---사용---
# dataset = S3StreamingDataset(img_path_df, BUCKET_NAME)
# loader = DataLoader(
#     dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=False,
#     num_workers=NUM_WORKERS,
#     collate_fn=collate_fn,
# )

# for i, batch in enumerate(tqdm(loader, desc="Streaming from S3")):
#     # 여기에 processor 적용 가능
#     print(f"Batch {i} - {len(batch['images'])} images loaded.")
#     print(f"Sample text: {batch['texts'][0][:100]}")
#     if i >= 2:  # 테스트용으로 3배치만
#         break

In [ ]:
if __name__ == "__main__":
    image_embeds, text_embeds, keys = extract_clip_embeddings(img_path_df)

    print("임베딩 완료!")
    print(f"이미지 임베딩 크기: {image_embeds.shape}")
    print(f"텍스트 임베딩 크기: {text_embeds.shape}")

    # 예시로 로컬에 저장
    torch.save({
        "keys": keys,
        "image_embeds": image_embeds,
        "text_embeds": text_embeds
    }, "/home/flexmatch_ftp/ftp/multimodal_categorizer/clip_embeddings.pt")

    print("💾 clip_embeddings.pt 저장 완료!")

##### multimodal classification model

In [33]:
category_labels = ['IT', '게임', '결혼/연애', '교육', '다이어트/건강보조식품', '만화/애니/툰', '문구/완구', '미술/디자인', '반려동물', '베이비/키즈', '뷰티', '브랜드공식계정',
                   '사진/영상', '셀럽', '스포츠', '시사', '엔터테인먼트', '여행/관광', '유명장소/핫플', '일상', '자동차/모빌리티', '짤/밈', '취미', '패션', '푸드', '홈/리빙']

In [36]:
len(category_labels)

26

In [46]:
label_map = {
    'IT': 0,
    '게임': 1,
    '결혼/연애': 2,
    '교육': 3,
    '다이어트/건강보조식품': 4,
    '만화/애니/툰': 5,
    '문구/완구': 6,
    '미술/디자인': 7,
    '반려동물': 8,
    '베이비/키즈': 9,
    '뷰티': 10,
    '브랜드공식계정': 11,
    '사진/영상': 12, # 원래는 사진/영상
    '셀럽': 13,
    '스포츠': 14,
    '시사': 15,
    '엔터테인먼트': 16,
    '여행/관광': 17,
    '유명장소/핫플': 18,
    '일상': 19,
    '자동차/모빌리티': 20,
    '짤/밈': 21,
    '취미': 22,
    '패션': 23,
    '푸드': 24,
    '홈/리빙': 25
}

img_path_df['label'] = img_path_df['main_category'].map(label_map)

In [48]:
img_path_df['label'] = img_path_df['label'].astype(int)

In [84]:
BUCKET_NAME = "flexmatch-data"
# TEXT_COLS = ["acnt_sub_nm_cleaned","intro_txt_cleaned","text"]
MODEL_NAME = "openai/clip-vit-base-patch32"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 128       
NUM_WORKERS = 8
EPOCHS = 3
LR = 2e-5
CHECKPOINT_PATH = "/home/flexmatch_ftp/ftp/multimodal_categorizer/clip_cls_ckpt.pt"
# ----------------------------

processor = CLIPProcessor.from_pretrained(MODEL_NAME, use_fast=False)  # or use_fast=True
clip_backbone = CLIPModel.from_pretrained(MODEL_NAME).to(DEVICE)

class S3StreamingDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    
    # 전체 데이터 길이 -> dataloader가 반복해야 할 전체 데이터 길이
    def __len__(self):
        return len(self.df)
    
    # Dataloader가 배치단위로 인덱스를 넘겨줄 때마다 그 때 해당 샘플만 s3에서 읽어옴
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        key = row["s3_path"]
        try:
            obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
            img = Image.open(BytesIO(obj['Body'].read())).convert("RGB")
        except Exception as e:
            img = Image.new("RGB",(224,224),(0,0,0))
        
        text_parts = []
        for col in ["acnt_sub_nm", "intro_txt", "media_text"]:
            if col in row and pd.notna(row[col]):
                text_parts.append(str(row[col]).strip())
        combined_text = ". ".join(text_parts) if text_parts else ""

        # parts = [str(row[c]).strip() for c in TEXT_COLS if c in row and pd.notna(row[c])]
        # text = ". ".join(parts) if parts else ""
        
        label = int(row['label'])
        return {"image": img, "text": combined_text, "label": label}

# 각 샘플들을 하나의 배치로 합치는 방법을 커스터마이징할 때 사용
# 해당 함수를 dataloader 인자로 넘겨주면 자동으로 batch 값을 넣어줌
# batch_size와 동일한 크기만큼의 샘플로 이루어진 하나의 배치를 생성하는 역할
# 그리고 그 processor로 텐서변환을 진행
def collate_fn(batch):
    images = [b["image"] for b in batch]
    texts  = [b["text"]  for b in batch]
    labels = torch.tensor([b["label"] for b in batch], dtype=torch.long)
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True, truncation=True)
    # move tensors later in loop to device
    inputs["labels"] = labels
    return inputs

# classifier wrapper
class CLIPMulticlassModel(nn.Module):
    def __init__(self, clip_model, num_classes, freeze_backbone=False):
        super().__init__()
        self.clip = clip_model
        proj = self.clip.config.projection_dim
        self.classifier = nn.Sequential(
            nn.Linear(proj*2, proj),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(proj, num_classes)
        )
        if freeze_backbone:
            for p in self.clip.parameters(): p.requires_grad = False
    def forward(self, input_ids=None, attention_mask=None, pixel_values=None):
        image_emb = self.clip.get_image_features(pixel_values=pixel_values)
        text_emb  = self.clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        image_emb = image_emb / image_emb.norm(dim=-1, keepdim=True)
        text_emb  = text_emb / text_emb.norm(dim=-1, keepdim=True)
        combined = torch.cat([image_emb, text_emb], dim=1)
        return self.classifier(combined)

# class CLIPMulticlassModel(nn.Module):
#     def __init__(self, clip_model, num_classes):
#         super().__init__()
#         self.clip = clip_model
#         self.classifier = nn.Linear(self.clip.config.projection_dim * 2, num_classes)

#     def forward(self, **inputs):
#         device = next(self.parameters()).device  
#         inputs = {k: v.to(device) if torch.is_tensor(v) else v for k, v in inputs.items()}

#         image_emb = self.clip.get_image_features(**inputs)
#         text_emb = self.clip.get_text_features(**inputs)

#         image_emb = image_emb / image_emb.norm(dim=-1, keepdim=True)
#         text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)

#         combined = torch.cat([image_emb, text_emb], dim=1)
#         return self.classifier(combined)


# -------- main training --------
def train(df):
    num_classes = len(category_labels)
    # batch_size만큼 데이터 샘플들을 배치 단위로 변경
    dataset = S3StreamingDataset(df)
    # 해당 dataset을 loader로 불러와서 텐서로 변환 진행
    # 따라서 dataloder는 샘플을 묶고 묶인 샘플을 최종 배치 텐서로 변환하는 역할을 진행
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True)
    model = CLIPMulticlassModel(clip_backbone, num_classes).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    global_step = 0
    for epoch in range(EPOCHS):
        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        model.train()
        for batch in pbar:
            # label을 제외한 모든 입력 텐서(input_ids, pixel_values, attention_mask)를 지정된 gpu 장치로 이동
            inputs = {k: v.to(DEVICE) for k,v in batch.items() if k != "labels"}
            labels = batch["labels"].to(DEVICE)

            # 순전파
            logits = model(**inputs)
            loss = criterion(logits, labels)

            # 최적화
            opt.zero_grad(); loss.backward(); opt.step()

            # 처리된 배치수를 업데이트
            global_step += 1
            pbar.set_postfix({"loss": float(loss.detach().cpu())})
            
            # gpu 메모리 최적화 및 관리를 위한 단계
            del inputs, labels, logits, loss
            gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()
        # checkpoint per epoch
        torch.save({"model": model.state_dict(), "opt": opt.state_dict(), "epoch": epoch}, CHECKPOINT_PATH)
    elapsed_hr = (time.time()-start)/3600
    print(f"Training done. time: {elapsed_hr:.2f} hr")

# if __name__ == "__main__":
#     train(img_path_df)


train, test split

In [55]:
BUCKET_NAME = "flexmatch-data"
MODEL_NAME = "openai/clip-vit-base-patch32"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 128       
NUM_WORKERS = 8
EPOCHS = 3
LR = 2e-5
CHECKPOINT_PATH = "/home/flexmatch_ftp/ftp/multimodal_categorizer/clip_cls_ckpt_train.pt"

def train(df):
    train_df, test_df = train_test_split(df)
    num_classes = len(category_labels)

    train_dataset = S3StreamingDataset(train_df)
    test_dataset = S3StreamingDataset(test_df)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True)

    model = CLIPMulticlassModel(clip_backbone, num_classes).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    global_step = 0
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [TRAIN]")
        for batch in pbar:
            inputs = {k: v.to(DEVICE) for k,v in batch.items() if k != "labels"}
            labels = batch["labels"].to(DEVICE)

            logits = model(**inputs)
            loss = criterion(logits, labels)

            opt.zero_grad(); loss.backward(); opt.step()

            train_loss += loss.item()
            global_step += 1
            pbar.set_postfix({"loss": float(loss.detach().cpu())})
            
            del inputs, labels, logits, loss
            gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()

        avg_train_loss = train_loss / len(train_loader)
        
        # 모델 평가 진행
        model.eval()
        test_loss, correct, total = 0, 0, 0

        with torch.no_grad():
            for batch in tqdm(test_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [EVAL]"):
                inputs = {k: v.to(DEVICE) for k, v in batch.items() if k != "labels"}
                labels = batch["labels"].to(DEVICE)

                logits = model(**inputs)
                loss = criterion(logits, labels)

                test_loss += loss.item()
                preds = torch.argmax(logits, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                del inputs, labels, logits, loss, preds
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

        avg_test_loss = test_loss / len(test_loader)
        acc = correct / total

        print(f"\nEpoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Test Loss={avg_test_loss:.4f}, Accuracy={acc*100:.2f}%\n")

        torch.save({
            "model": model.state_dict(),
            "opt": opt.state_dict(),
            "epoch": epoch,
            "train_loss": avg_train_loss,
            "test_loss": avg_test_loss,
            "accuracy": acc
        }, CHECKPOINT_PATH)

        elapsed_hr = (time.time() - start) / 3600
        print(f"Training done. Total time: {elapsed_hr:.2f} hr")

        # checkpoint per epoch
        torch.save({"model": model.state_dict(), "opt": opt.state_dict(), "epoch": epoch}, CHECKPOINT_PATH)

    return model

if __name__ == "__main__":
    train(img_path_df)

Epoch 1/3 [TRAIN]:   0%|          | 0/774 [00:00<?, ?it/s]

Epoch 1/3 [EVAL]: 100%|██████████| 258/258 [17:37<00:00,  4.10s/it]



Epoch 1: Train Loss=1.9079, Test Loss=0.9325, Accuracy=86.90%

Training done. Total time: 1.17 hr


Epoch 2/3 [EVAL]: 100%|██████████| 258/258 [16:54<00:00,  3.93s/it]



Epoch 2: Train Loss=0.5930, Test Loss=0.3910, Accuracy=95.85%

Training done. Total time: 2.30 hr


Epoch 3/3 [EVAL]: 100%|██████████| 258/258 [16:27<00:00,  3.83s/it]



Epoch 3: Train Loss=0.2585, Test Loss=0.2202, Accuracy=97.06%

Training done. Total time: 3.40 hr


no labeling data inference

In [86]:
class S3StreamingInferenceDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        key = row["s3_path"]
        try:
            obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=key)
            img = Image.open(BytesIO(obj['Body'].read())).convert("RGB")
        except Exception:
            img = Image.new("RGB", (224,224), (0,0,0))
        
        text_parts = []
        for col in ["acnt_sub_nm", "intro_txt", "media_text"]:
            if col in row and pd.notna(row[col]):
                text_parts.append(str(row[col]).strip())
        combined_text = ". ".join(text_parts) if text_parts else ""

        return {"image": img, "text": combined_text}


def collate_fn_predict(batch):
    texts = [b["text"] for b in batch]
    images = [b["image"] for b in batch]
    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    return {k: v.to(DEVICE) for k, v in inputs.items()}

def predict(df, model):
    model.eval()
    ds = S3StreamingInferenceDataset(df)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_predict)

    preds = []
    with torch.no_grad():
        for inputs in tqdm(loader, desc="Predicting"):
            # 디바이스 상태 확인용 디버깅 로그 추가
            for k, v in inputs.items():
                if isinstance(v, torch.Tensor):
                    print(f"{k}: {v.device}")  # ✅ 어디에 있는지 확인
                    inputs[k] = v.to(DEVICE, non_blocking=True)

            model_device = next(model.parameters()).device
            print(f"Model device: {model_device}")  # ✅ 모델 디바이스도 확인

            logits = model(**inputs)
            pred = torch.argmax(logits, dim=1).cpu().numpy()
            preds.extend(pred)
            break  # 일단 한 batch만 확인

    return preds



In [88]:
if __name__=="__main__":
    CHECKPOINT_PATH = "/home/flexmatch_ftp/ftp/multimodal_categorizer/clip_cls_ckpt_train.pt"
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    
    num_classes = len(category_labels)
    clip_backbone = CLIPModel.from_pretrained(MODEL_NAME).to(DEVICE)
    model = CLIPMulticlassModel(clip_backbone, num_classes)

    model.load_state_dict(checkpoint["model"])
    model.to(DEVICE)
    model.eval()

    # test할 label 없는 새로운 데이터
    train_df, test_df = train_test_split(img_path_df, test_size=0.2, random_state=42)
    new_test_df = test_df[:100]
    new_test_df.drop(['label'], axis=1, inplace=True)
    
    predictions = predict(new_test_df, model)
    new_test_df['pred_label'] = predictions
    new_test_df['pred_category'] = new_test_df['pred_label'].map({i: labels for i, labels in enumerate(category_labels)})

    new_test_df.to_csv("/home/flexmatch_ftp/ftp/multimodal_categorizer/inference_test.csv")
    print("예측 완료!")


/tmp/ipykernel_2534/371550618.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df.drop(['label'], axis=1, inplace=True)
Predicting:   0%|          | 0/1 [00:22<?, ?it/s]

pixel_values: cuda:0
input_ids: cuda:0
attention_mask: cuda:0
Model device: cuda:0
예측 완료!



/tmp/ipykernel_2534/371550618.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df['pred_label'] = predictions
/tmp/ipykernel_2534/371550618.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df['pred_category'] = new_test_df['pred_label'].map({i: labels for i, labels in enumerate(category_labels)})
